In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [123]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 9

5


In [124]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=573, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-12)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()'],
 ["['avg_sub_att_dif', 'empty_arena', 'B_current_lose_streak', 'R_win_by_Decision_Split', 'R_Flyweight_rank', 'B_Welterweight_rank', 'R_Heavyweight_rank', 'R_current_win_streak', 'B_Light Heavyweight_rank', 'win_streak_dif', 'R_win_by_TKO_Doctor_Stop

In [125]:
df = pd.read_csv("../data/ufc-master.csv")

In [126]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [127]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [128]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [129]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4155
4155
4155


In [130]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Weili Zhang,Joanna Jedrzejczyk,-185,170,54.054054,170.00000,2020-03-07,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
201,Beneil Dariush,Drakkar Klose,-162,152,61.728395,152.00000,2020-03-07,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
202,Neil Magny,Li Jingliang,157,-167,157.000000,59.88024,2020-03-07,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
203,Alex Oliveira,Max Griffin,-125,115,80.000000,115.00000,2020-03-07,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
204,Sean O'Malley,Jose Quinonez,-330,300,30.303030,300.00000,2020-03-07,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4350,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.00000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1
4351,John Howard,Daniel Roberts,-210,175,47.619048,175.00000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0
4352,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.00000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4353,Mike Pierce,Julio Paulino,-420,335,23.809524,335.00000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Anthony Smith,Aleksandar Rakic,225,-278,225.000000,35.971223,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Robbie Lawler,Neil Magny,200,-250,200.000000,40.000000,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Ji Yeon Kim,Alexa Grasso,230,-315,230.000000,31.746032,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Ricardo Lamas,Bill Algeo,-335,240,29.850746,240.000000,2020-08-29,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Maki Pitolo,Impa Kasanganay,115,-139,115.000000,71.942446,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Elizeu Dos Santos,Aleksei Kunchenko,-122,112,81.967213,112.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
196,Mayra Bueno Silva,Maryna Moroz,-140,130,71.428571,130.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,...,8.0,0.00,0.25,0.0,0.0,0.0,1.0,0.0,0.0,1
197,Bruno Silva,David Dvorak,-110,100,90.909091,100.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,...,1.0,0.27,0.00,0.0,0.0,2.0,1.0,0.0,2.0,1
198,Veronica Macedo,Bea Malecki,-155,145,64.516129,145.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,...,0.0,0.00,0.00,0.0,0.0,0.0,2.0,0.0,0.0,1


In [131]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [132]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Anthony Smith,Aleksandar Rakic,225,-278,225.000000,35.971223,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Robbie Lawler,Neil Magny,200,-250,200.000000,40.000000,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Ji Yeon Kim,Alexa Grasso,230,-315,230.000000,31.746032,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Ricardo Lamas,Bill Algeo,-335,240,29.850746,240.000000,2020-08-29,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Maki Pitolo,Impa Kasanganay,115,-139,115.000000,71.942446,2020-08-29,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Elizeu Dos Santos,Aleksei Kunchenko,-122,112,81.967213,112.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
196,Mayra Bueno Silva,Maryna Moroz,-140,130,71.428571,130.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,...,8.0,0.00,0.25,0.0,0.0,0.0,1.0,0.0,0.0,1
197,Bruno Silva,David Dvorak,-110,100,90.909091,100.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,...,1.0,0.27,0.00,0.0,0.0,2.0,1.0,0.0,2.0,1
198,Veronica Macedo,Bea Malecki,-155,145,64.516129,145.000000,2020-03-14,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,...,0.0,0.00,0.00,0.0,0.0,0.0,2.0,0.0,0.0,1


In [133]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [134]:
models[1][model_num]

'GaussianNB()'

In [135]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [136]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [137]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [138]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [139]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_7_25_20
GaussianNB()
['R_Reach_cms', 'total_round_dif', 'R_Height_cms', 'R_avg_SIG_STR_pct', 'B_age', 'R_longest_win_streak', 'lose_streak_dif', 'ko_dif', 'R_win_by_Decision_Majority', 'longest_win_streak_dif', 'avg_sub_att_dif', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
1


In [140]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [141]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [21]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 20024.23616321908
Feature: B_avg_TD_landed Score: 20531.37026498167
Feature: B_total_rounds_fought Score: 21318.41055249737
Feature: R_avg_TD_pct Score: 22225.3897094801
Feature: B_Lightweight_rank Score: 22452.640320617294
The best feature was B_Lightweight_rank.  It scored 22452.640320617294
Current best score is: 22452.640320617294
Feature: B_current_lose_streak Score: 22537.784313525797
Feature: B_longest_win_streak Score: 23051.526758344113
Feature: B_total_rounds_fought Score: 23233.395513538853
Feature: R_age Score: 23992.502573711445
Feature: R_Light Heavyweight_rank Score: 25452.432036189373
Feature: B_Light Heavyweight_rank Score: 25821.902714869433
The best feature was B_Light Heavyweight_rank.  It scored 25821.902714869433
Current best score is: 25821.902714869433
Feature: win_streak_dif Score: 25905.84671298658
The best feature was win_streak_dif.  It scored 25905.84671298658
Current best score is: 25905.84671298658
NO IMPROVEMENT
FINAL BEST SCORE: 2

(3212, 27)

(3212,)

(3212, 2)

(200, 27)

...

(200, 24)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,225,-278,0.999735,0.000265,1
1,200,-250,0.089152,0.910848,1
2,230,-315,0.232562,0.767438,1
3,-335,240,0.908442,0.091558,0
4,115,-139,0.588590,0.411410,1
...,...,...,...,...,...
195,-122,112,0.851697,0.148303,0
196,-140,130,0.884861,0.115139,1
197,-110,100,0.697492,0.302508,1
198,-155,145,0.999462,0.000538,1



          Number of matches: 200
          Number of bets: 179
          Number of winning bets: 114
          Number of losing bets: 65
          Number of underdog bets: 71
          Number of underdog wins: 33
          Number of underdog losses: 38
          Number of Favorite bets: 101
          Number of favorite wins: 77
          Number of favorite losses: 24
          Number of even bets: 7
          Number of even wins: 4
          Number of even losses: 3
          Profit: 2581.501543066843
          Profit per bet: 14.421796329982364
          Profit per match: 12.907507715334216
          
          
HI
HI
HI
HI
HI


In [142]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3212, 22)

(3212,)

(3212, 2)

(200, 22)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,225,-278,0.436896,0.563104,1
1,200,-250,0.107175,0.892825,1
2,230,-315,0.227217,0.772783,1
3,-335,240,0.910745,0.089255,0
4,115,-139,0.569137,0.430863,1
...,...,...,...,...,...
195,-122,112,0.866733,0.133267,0
196,-140,130,0.906298,0.093702,1
197,-110,100,0.703606,0.296394,1
198,-155,145,0.999632,0.000368,1



          Number of matches: 200
          Number of bets: 187
          Number of winning bets: 118
          Number of losing bets: 69
          Number of underdog bets: 74
          Number of underdog wins: 34
          Number of underdog losses: 40
          Number of Favorite bets: 104
          Number of favorite wins: 78
          Number of favorite losses: 26
          Number of even bets: 9
          Number of even wins: 6
          Number of even losses: 3
          Profit: 2274.342545429834
          Profit per bet: 12.162259601229058
          Profit per match: 11.37171272714917
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [143]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [144]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [145]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [146]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=573, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-12)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()'],
 ["['avg_sub_att_dif', 'empty_arena', 'B_current_lose_streak', 'R_win_by_Decision_Split', 'R_Flyweight_rank', 'B_Welterweight_rank', 'R_Heavyweight_rank', 'R_current_win_streak', 'B_Light Heavyweight_rank', 'win_streak_dif', 'R_win_by_TKO_Doctor_Stop